In [3]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error

import warnings
# Ignore warnings
warnings.filterwarnings("ignore")

# Prepare dataset
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

def walk_forward_cv(data, window_size=10, n_splits=5, epochs=50):
    fold_size = (len(data) - window_size) // (n_splits + 1)
    errors = []

    for i in range(n_splits):
        end_train = window_size + fold_size * (i + 1)
        train_data = data[:end_train]
        val_data = data[end_train:end_train + fold_size]

        # Skip if not enough validation data
        if len(val_data) < 1:
            break

        # Create sequences
        X_train, y_train = create_sequences(train_data, window_size)
        X_val, y_val = create_sequences(np.concatenate([train_data[-window_size:], val_data]), window_size)

        # Reshape for LSTM
        X_train = X_train[..., np.newaxis]
        X_val = X_val[..., np.newaxis]

        # Build model
        model = tf.keras.Sequential([
            tf.keras.layers.LSTM(128, input_shape=(window_size, 1)),
            tf.keras.layers.Dense(1)
        ])
        model.compile(loss='mse', optimizer='adam')
        model.fit(X_train, y_train, epochs=epochs, verbose=0)

        # Evaluate
        y_pred = model.predict(X_val, verbose=0).flatten()
        error = mean_squared_error(y_val, y_pred)
        errors.append(error)
        print(f"Fold {i+1}/{n_splits}, MSE: {error:.6f}")

    print(f"\nAverage CV MSE: {np.mean(errors):.6f}")
    return errors

# Example usage:
# voltage_data = np.array([...])  # Your 104 measurements
# walk_forward_cv(voltage_data, window_size=10, n_splits=5, epochs=50)

voltage_data = np.array([69.03, 71.27, 73.27, 76.2, 77.57, 79.14, 81.14, 83.14, 85.28, 86.79, 87.49, 89.91, 90.91, 92.8, 94.74, 96.94, 97.98,
    99.28, 102.29, 103.23, 104.25, 105.25, 106.43, 107.14, 108, 109.03, 110.25, 112.82, 113.91, 115.07, 116.35, 117.98, 118.91,
    119.19, 120.48, 121.28, 122.68, 123.16, 124.91, 125.81, 127.32, 128.62, 130.1, 131.02, 131.92, 132.76, 133.78, 134.7, 135.62,
    136.38, 136.83, 137.24, 138.15, 139.05, 139.48, 139.83, 140.48, 141.48, 142.75, 143.22, 143.42, 144.62, 145.83, 147.03, 148.56,
    149.1, 149.18, 150.55, 150.95, 151.35, 151.75, 152.15, 152.74, 153.34, 153.94, 154.53, 154.6, 155.02, 155.45, 156.78, 155.35, 156.1,
    156.85, 159.14, 157.33, 158.38, 159.43, 160.48, 161.53, 162.58, 163.03, 163.49, 164.45, 163.76, 164.59, 165.43, 166.26, 166.04, 166.61,
    167.17, 167.73, 168.3, 168.86, 169.9])

walk_forward_cv(voltage_data, window_size=10, n_splits=5, epochs=50)


Fold 1/5, MSE: 11474.339551
Fold 2/5, MSE: 16033.363415
Fold 3/5, MSE: 16197.156291
Fold 4/5, MSE: 18154.544127
Fold 5/5, MSE: 19076.448965

Average CV MSE: 16187.170470


[11474.339551248504,
 16033.36341502776,
 16197.15629110991,
 18154.544127001824,
 19076.44896510004]